In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using DrakeVisualizer
using Plots
using RigidBodyTreeInspector

using BilevelTrajOpt
gr(size=(500,500), html_output_format=:png);

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9dd84cf97c4fc16956f39829c81f3b6907c8fc83-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9dd84cf97c4fc16956f39829c81f3b6907c8fc83-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
env = parse_contacts(mechanism, urdf, .5);
x0 = MechanismState(mechanism)
Δt = 0.01;

In [3]:
q0 = [1., 0., 0., 0., 0., 0., .025]
v0 = [0., 0., 0., .25, 0. , 0.]
N = 10

set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)

println("Explicit Contacts")
traj_exp = BilevelTrajOpt.simulate(x0,env,Δt,N,implicit_contact=false);
println("Implicit Contacts")
traj_imp = BilevelTrajOpt.simulate(x0,env,Δt,N,implicit_contact=true);

Explicit Contacts


6-element Array{Float64,1}:
 -4.11889e-40
  8.27181e-25
  5.9533e-52 
  7.12718e-7 
 -7.14088e-22
 -2.23189e-6 

20-element Array{Float64,1}:
  1.0        
  5.33477e-44
 -4.17619e-53
  4.3649e-49 
  0.0025     
  7.10673e-26
  0.024019   
  1.06695e-41
 -5.34553e-51
  8.72981e-47
  0.25       
  7.10673e-24
 -0.0981     
  0.0        
  3.38564e-9 
  1.14489e-8 
  7.60378e-8 
  1.14489e-8 
  0.252931   
  2.27511e-7 


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Solve_Succeeded


6-element Array{Float64,1}:
  4.03428e-40
  3.30872e-24
  3.18481e-51
  1.35271e-6 
  5.84254e-22
 -3.47235e-6 

...

20-element Array{Float64,1}:
  1.0        
 -8.01513e-44
 -3.30872e-28
 -1.24315e-45
  0.016266   
  1.51132e-19
 -1.96562e-9 
 -9.0404e-42 
 -2.86986e-41
 -3.50506e-44
 -4.44246e-9 
  5.14179e-22
  2.12264e-7 
  0.0        
  0.17398    
  0.0495435  
  0.17398    
  0.0495435  
  3.42896e-8 
  1.0        

6-element Array{Float64,1}:
  0.0        
  0.0        
 -4.09862e-44
 -4.44362e-7 
  2.45943e-42
 -9.81001    

20-element Array{Float64,1}:
  1.0        
 -1.25353e-43
 -3.30872e-28
 -1.41636e-45
  0.016266   
  1.51126e-19
  8.65291e-10
 -9.0404e-42 
 -2.86986e-41
 -3.46408e-44
  1.15344e-12
 -5.88014e-22
  2.83091e-7 
  0.0        
  0.17399    
  0.0495334  
  0.17399    
  0.0495334  
  2.71512e-8 
  1.0        

Solve_Succeeded
Solve_Succeeded
Implicit Contacts


LoadError: [91mUndefVarError: num_contacts not defined[39m

In [ ]:
ts = cumsum([Δt for i in 1:N]);
plot(traj_exp[11,:],label="explicit")
plot!(traj_imp[11,:],label="implicit")

In [ ]:
# display(traj_exp[1:7,8])
# display(traj_exp[8:13,8])
display(traj_exp[1:7,9])
display(traj_exp[8:13,9])

In [ ]:
# display(traj_imp[1:7,8])
# display(traj_imp[8:13,8])
display(traj_imp[1:7,9])
display(traj_imp[8:13,9])

In [ ]:
display(traj_exp[8:13,5:9])
display(traj_imp[8:13,5:9])

In [ ]:
println(traj_exp[7+6+1:end,9])
# println(traj_imp[7+6+1:end,9])

In [ ]:
display("explicit")
display(traj_exp[14:end,9])
display("implicit")
display(traj_imp[14:end,9])

In [ ]:
println(traj_imp[14:end,9])

In [ ]:
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1));
geometries = visual_elements(mechanism, URDFVisuals(urdf))
vis = Visualizer(mechanism, geometries);

In [ ]:
traj = traj_exp
qs = [traj[1:num_positions(x0),i] for i in 1:N]
ts = cumsum([Δt for i in 1:N]);
RigidBodyTreeInspector.animate(vis, mechanism, ts, qs; realtimerate = .25);